# qm_geo_opt — Run quantum accurate geometry optimisation on a small molecule

See [the tutorial](/Tutorials/qm_geo_opt_calculation.ipynb).

In [ ]:
import os
from glob import glob
from pathlib import Path
import json

import rush

# Define our project information
EXPERIMENT = "rush qm geo opt notebook"
MOLECULE_NAME = "cyclobutane"
TAGS = ["qdx", EXPERIMENT, MOLECULE_NAME]

# |hide_line
WORK_DIR = Path.home() / "qdx" / EXPERIMENT

# |hide_line
if WORK_DIR.exists():
# |hide_line
    client = rush.Provider(workspace=WORK_DIR)
# |hide_line
    await client.nuke(remote=False)
# |hide_line
os.makedirs(WORK_DIR, exist_ok=True)

# Get our client, for calling modules and using the rush API
# By using the `build_provider_with_functions` method, we will also build
# helper functions calling each module
# - edit in your access token and API URL or set the RUSH_TOKEN and RUSH_URL environment variables
client = await rush.build_provider_with_functions(
    access_token=os.getenv("RUSH_TOKEN"),
    url=os.getenv("RUSH_URL"),
    batch_tags=TAGS,
)

# |hide_line
client = await rush.build_provider_with_functions(batch_tags=TAGS, workspace=WORK_DIR, restore_by_default=True)

# RUSH_TOKEN and RUSH_URL are discovered automatically if present in the env.
# Alternatively, they can be set explicitly:
# TOKEN = os.getenv("RUSH_TOKEN")
# URL = os.getenv("RUSH_URL")
# client = await rush.build_provider_with_functions(access_token=TOKEN, url=URL, batch_tags=TAGS)

# Get our latest modules as a dict[module_name, module_path]
# If a lock file exists, load it so that the run is reproducible
# This will be done automatically if you use the `build_provider_with_functions`
# method
modules = await client.get_latest_module_paths()

# Create input geometry
def xyz_to_qdxf_topology(xyz_data: str):
    symbols = []
    geometry = []
    for line in xyz_data.splitlines()[2:]:
        symbol, x, y, z = line.split()
        symbols.append(symbol)
        geometry.extend([float(x), float(y), float(z)])
    return {"topology": {"symbols": symbols, "geometry": geometry}}


def qdxf_topology_to_xyz(qdxf_in: dict, name: str):
    symbols = qdxf_in["topology"]["symbols"]
    geometry = qdxf_in["topology"]["geometry"]
    xyz_data = f"{len(symbols)}\n{name}\n"
    for i in range(len(symbols)):
        symbol = symbols[i]
        x, y, z = geometry[3 * i : 3 * i + 3]
        xyz_data += f"  {symbol}"
        for coord in [x, y, z]:
            xyz_data += "     "
            if coord >= 0:
                xyz_data += " "
            xyz_data += f"{(coord):.4f}"
        xyz_data += "\n"
    return xyz_data

cyclobutane_xyz = """12
cyclobutane
  C     -0.750      0.452     -0.417
  C     -0.696     -0.588      0.609
  C      0.820     -0.678      0.537
  C      0.892      0.417     -0.428
  H     -1.285      1.273      0.066
  H     -1.328      0.080     -1.263
  H     -1.225     -1.507      0.366
  H     -1.029     -0.162      1.555
  H      1.158     -1.594      0.054
  H      1.310     -0.477      1.488
  H      1.432      0.009     -1.290
  H      1.506      1.236     -0.056
"""

cyclobutane_qdxf = xyz_to_qdxf_topology(cyclobutane_xyz)

# write the qdxf file to disk
cyclobutane_qdxf_input_file = Path(WORK_DIR) / "cyclobutane.qdxf.json"
with open(cyclobutane_qdxf_input_file, "w") as f:
    json.dump(cyclobutane_qdxf, f)

# Arguments
params = {
    "use_internal_coords": True,
    "max_iterations": 100,
    "conv_threshold": 1e-4,
    "basis_set": "cc-pVDZ",
}

# Call geometry optimisation over the input geometry
QM_GEO_OPT_RESOURCES = {
    "gpus": 1,
    "storage": 1024_000,
    "walltime": 60,
}
(geo_opt_out, results) = await client.qm_geo_opt(
    cyclobutane_qdxf_input_file, params, resources=QM_GEO_OPT_RESOURCES
)
print(await geo_opt_out.get())

# Check that calculations converged
results_out = await results.get()
assert results_out["converged"]

# Visualise pre-optimised and optimised geometries
import py3Dmol

pre_opt_view = py3Dmol.view()
pre_opt_view.addModel(cyclobutane_xyz, "xyz")
pre_opt_view.setStyle({"stick": {}})
pre_opt_view.zoomTo()
pre_opt_view.show()

topologies_out = await geo_opt_out.get()
opt_cyclobutane_xyz = qdxf_topology_to_xyz(
    topologies_out[-1], "optimised cyclobutane"
)

opt_view = py3Dmol.view()
opt_view.addModel(opt_cyclobutane_xyz, "xyz", {})
opt_view.setStyle({"stick": {}})
opt_view.zoomTo()
opt_view.show()